<a href="https://colab.research.google.com/github/andersthequeen/cis545/blob/master/538_Presidential_Election_Confidence_Interval_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#538 Presidential Election Confidence Interval Review

The purpose of this short piece of code is to scrape from NYT's election data, discern the voting percentages of the incumbent (Trump) and challenger (Biden), and evaluate how 538's projected confidence intervals lined up with the true results of the 2020 Presidential Election.


#Scraping


In [26]:
%%capture
!pip3 install requests_html

import re
import urllib
import requests
import pandas as pd
from lxml import html
from lxml import etree
from io import StringIO, BytesIO
from requests_html import HTML, HTMLSession
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [149]:
session = HTMLSession()
election_urls = []
scrape_df = pd.DataFrame(columns = ['state', 'd_real', 'r_real'])
url = "https://www.nytimes.com/interactive/2020/11/03/us/elections/results-president.html?action=click&pgtype=Article&state=default&module=styln-elections-2020&region=TOP_BANNER&context=election_recirc"
r = session.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

for a in soup.find_all('a', {'class': 'e-text-color e-democrat'}):
  election_urls.append(a['href'])
for b in soup.find_all('a', {'class': 'e-text-color e-republican'}):
  election_urls.append(b['href'])
election_urls = list(dict.fromkeys(election_urls))

for url in election_urls:
  r = session.get(url)
  soup = BeautifulSoup(r.text, 'html.parser')
  party = soup.findAll("span", attrs = {"class": "e-party-display"})
  percent = soup.findAll("span", attrs = {"class": "e-percent-val"})

  if(str(party[0].contents[0]) == "Democrat"):
    d_real = float(percent[0].contents[0])
    r_real = float(percent[1].contents[0])
  else:
    d_real = float(percent[1].contents[0])
    r_real = float(percent[0].contents[0])
  
  state = str(url).split('results-')[1].split('-president')[0]

  scrape_df = scrape_df.append({'state' : state, 'd_real' : d_real, 'r_real' : r_real}, ignore_index = True)

scrape_df

,state,d_real,r_real
0,california,64.60,33.40
1,connecticut,59.30,39.10
2,delaware,58.80,39.80
3,district-of-columbia,92.60,5.20
4,hawaii,63.70,34.30
5,illinois,55.40,42.70
6,maryland,63.50,34.60
7,massachusetts,65.30,32.50
8,new-jersey,58.40,40.10
9,new-mexico,54.20,43.60


In [143]:
# Import the 538 Confidence Interval Data


 , i'm a beginner 


In [ ]:
"import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Setup a plot such that only the bottom spine is shown
def setup(ax):
    ax.spines['right'].set_color('none')
    ax.spines['left'].set_color('none')
    ax.yaxis.set_major_locator(ticker.NullLocator())
    ax.spines['top'].set_color('none')
    ax.xaxis.set_ticks_position('bottom')
    ax.tick_params(which='major', width=1.00)
    ax.tick_params(which='major', length=5)
    # ax.tick_params(which='minor', width=0.75)
    # ax.tick_params(which='minor', length=2.5)
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 1)
    ax.patch.set_alpha(0.0)

In [ ]:
for x in range(5):
  plt.figure(figsize=(8, 6))
  n = 8
  
  ax = plt.subplot(n, 1, 1)
  setup(ax)
  ax.xaxis.set_major_locator(ticker.MultipleLocator(10))
  ax.plot([10, 20, 30], [0.2, 0.2, 0.2], 'bo')
  ax.axvspan(xmin = 30, xmax = 40, ymax = 0.5, alpha=0.3, color='red')
  ax.text(0, -0.9, "California", fontsize=14,transform=ax.transAxes)
  
  # Push the top of the top axes outside the figure because we only show the
  # bottom spine.
  plt.subplots_adjust(left=0.05, right=0.95, bottom=0.05, top=1.05)
  plt.show()
